In [ ]:
!pip install transformers datasets
!pip install accelerate
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


# Fine-tune a pretrained model

There are significant benefits to using a pretrained model. It reduces computation costs, your carbon footprint, and allows you to use state-of-the-art models without having to train one from scratch. 🤗 Transformers provides access to thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training technique. In this tutorial, you will fine-tune a pretrained model with a deep learning framework of your choice:

* Fine-tune a pretrained model with 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer).
* Fine-tune a pretrained model in TensorFlow with Keras.
* Fine-tune a pretrained model in native PyTorch.

<a id='data-processing'></a>

## Prepare a dataset

Before you can fine-tune a pretrained model, download a dataset and prepare it for training.

In [ ]:
!rm -fr csic-dataset
!rm -fr *.jsonl
!rm -fr *.rar
!rm -fr *.txt
!rm -fr test_trainer

In [ ]:
!wget --no-check-certificate https://www.isi.csic.es/dataset/normalTrafficTraining.rar
!wget --no-check-certificate https://www.isi.csic.es/dataset/normalTrafficTest.rar
!wget --no-check-certificate https://www.isi.csic.es/dataset/anomalousTrafficTest.rar

--2023-12-18 18:40:23--  https://www.isi.csic.es/dataset/normalTrafficTraining.rar
Resolving www.isi.csic.es (www.isi.csic.es)... 161.111.32.51
Connecting to www.isi.csic.es (www.isi.csic.es)|161.111.32.51|:443... connected.
	requested host name ‘www.isi.csic.es’.
HTTP request sent, awaiting response... 200 OK
Length: 1225993 (1.2M) [application/octet-stream]
Saving to: ‘normalTrafficTraining.rar’

normalTrafficTraini 100%[===================>]   1.17M   491KB/s    in 2.4s    

2023-12-18 18:40:26 (491 KB/s) - ‘normalTrafficTraining.rar’ saved [1225993/1225993]

--2023-12-18 18:40:27--  https://www.isi.csic.es/dataset/normalTrafficTest.rar
Resolving www.isi.csic.es (www.isi.csic.es)... 161.111.32.51
Connecting to www.isi.csic.es (www.isi.csic.es)|161.111.32.51|:443... connected.
	requested host name ‘www.isi.csic.es’.
HTTP request sent, awaiting response... 200 OK
Length: 1228056 (1.2M) [application/octet-stream]
Saving to: ‘normalTrafficTest.rar’

normalTrafficTest.r 100%[============

In [ ]:
!unrar x anomalousTrafficTest.rar
!unrar x normalTrafficTest.rar
!unrar x normalTrafficTraining.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from anomalousTrafficTest.rar

Extracting  anomalousTrafficTest.txt                                       3%  7% 10% 14% 17% 21% 24% 28% 31% 35% 38% 42% 46% 49% 53% 56% 60% 63% 67% 70% 74% 77% 81% 85% 88% 92% 95% 99%100%  OK 
All OK

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from normalTrafficTest.rar

Extracting  normalTrafficTest.txt                                          2%  5%  8% 10% 13% 16% 18% 21% 24% 26% 29% 32% 34% 37% 39% 42% 45% 47% 50% 53% 55% 58% 61% 63% 66% 69% 71% 74% 77% 79% 82% 85% 87% 90% 93% 95% 98%100%  OK 
All OK

UNRAR 6.11 beta 1 freeware      Copyright (c) 19

In [ ]:
import re
from urllib.parse import unquote

In [ ]:
def text_to_dict(fname, label):
  with open(fname, 'r') as f:
    log_data = f.read()

    # Regular expression patterns for matching GET and POST requests along with bodies
    get_pattern = r"(GET .*? HTTP/1\.[01]\n(.*?))(?=\n\n|\Z)"
    post_pattern = r"(POST .*? HTTP/1\.[01]\n(.*?)\n\n(.*?))(?=\n\n|\Z)"

    # Extract HTTP GET requests
    get_requests = re.findall(get_pattern, log_data, re.DOTALL)

    # Extract HTTP POST requests
    post_requests = re.findall(post_pattern, log_data, re.DOTALL)

    payload = []

    for req in get_requests[:5]:
        if '?' in req[0].split('\n')[0]:
          query_params = req[0].split('\n')[0].split('?')[-1]

          payload_dict = {}
          payload_dict['method'] = 'GET'
          payload_dict['payload'] = query_params
          payload_dict['label'] = label
          payload.append(payload_dict)

    for req in post_requests:
        query_params = req[0].split('\n')[-1]

        payload_dict = {}
        payload_dict['method'] = 'POST'
        payload_dict['payload'] = query_params
        payload_dict['label'] = label
        payload.append(payload_dict)

    return payload

In [ ]:
from tqdm import tqdm

datafiles = {
    'anomalousTrafficTest.txt': 'Anomalous',
    'normalTrafficTest.txt': 'Normal',
    'normalTrafficTraining.txt': 'Normal',
}

payload = []

for fname, label in tqdm(datafiles.items()):
  payload.extend(text_to_dict(fname, label))

100%|██████████| 3/3 [00:05<00:00,  1.88s/it]


In [ ]:
payload[0]

{'method': 'GET',
 'payload': 'id=2&nombre=Jam%F3n+Ib%E9rico&precio=85&cantidad=%27%3B+DROP+TABLE+usuarios%3B+SELECT+*+FROM+datos+WHERE+nombre+LIKE+%27%25&B1=A%F1adir+al+carrito HTTP/1.1',
 'label': 'Anomalous'}

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(payload)
df

,method,payload,label
0,GET,id=2&nombre=Jam%F3n+Ib%E9rico&precio=85&cantid...,Anomalous
1,GET,id=2%2F&nombre=Jam%F3n+Ib%E9rico&precio=85&can...,Anomalous
2,GET,modo=entrar&login=bob%40%3CSCRipt%3Ealert%28Pa...,Anomalous
3,GET,modo=entrar&login=grimshaw&pwd=G%2F%2FlAc%2CIA...,Anomalous
4,POST,id=2&nombre=Jam%F3n+Ib%E9rico&precio=85&cantid...,Anomalous
...,...,...,...
25585,POST,errorMsg=Credenciales+incorrectas,Normal
25586,POST,modo=insertar&precio=6460&B1=Pasar+por+caja,Normal
25587,POST,modo=registro&login=xaviera&password=och*avero...,Normal
25588,POST,B2=Vaciar+carrito,Normal


In [ ]:
# Randomly shuffle the index
shuffled_index = df.sample(frac=1).index

# Split the DataFrame into two based on a specified ratio (e.g., 70-30 split)
split_ratio = 0.7
split_index = int(split_ratio * len(df))

# Create two DataFrames using the shuffled index
train_df = df.loc[shuffled_index[:split_index]]
test_df = df.loc[shuffled_index[split_index:]]

In [ ]:
train_df.to_json(f"payload_train.jsonl", orient="records", lines=True)
test_df.to_json(f"payload_test.jsonl", orient="records", lines=True)

### Prepare CSIC Dataset

In [ ]:
from datasets import load_dataset

In [ ]:
data_files = {"train" : "payload_train.jsonl", "test" : "payload_test.jsonl"}
csic_dataset = load_dataset("json", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
csic_dataset

DatasetDict({
    train: Dataset({
        features: ['method', 'payload', 'label'],
        num_rows: 17913
    })
    test: Dataset({
        features: ['method', 'payload', 'label'],
        num_rows: 7677
    })
})

In [ ]:
csic_dataset["train"][:5]

{'method': ['POST', 'POST', 'POST', 'POST', 'POST'],
 'payload': ['modo=registro&login=snips&password=impenetr-abilidad&nombre=Kalanit&apellidos=Pin%F3s+Turrubiartes&email=ornellas3%40professionalhost.com.pro&dni=98057218F&direccion=Calle+Batalla+De+Otumba+26%2C+&ciudad=Lora+de+Estepa&cp=43120&provincia=Salamanca&ntc=3586735536952135&B1=Registrar',
  'idA=2&nombre=Vino+Rioja&precio=39&cantidad=24&B1=A%F1adir+al+carrito',
  'id=1',
  'id=2&nombre=Vino+Rioja&precio=100&cantidad=2&B1=A%F1adir+al+carrito',
  'id=3&nombre=Vino+Rioja&precio=100&cantidad=46&B1=A%F1adir+al+carrito'],
 'label': ['Normal', 'Anomalous', 'Normal', 'Normal', 'Normal']}

In [ ]:
csic_dataset["test"][:5]

{'method': ['POST', 'POST', 'POST', 'POST', 'POST'],
 'payload': ['errorMsg=Credenciales+incorrectas',
  'id=2',
  'B2=Vaciar+carrito',
  'B2=Vaciar+carrito',
  'modo=insertar&precio=1565&B1=Pasar+por+caja'],
 'label': ['Normal', 'Normal', 'Normal', 'Normal', 'Normal']}

### Cleaning up the data

In [ ]:
csic_dataset = csic_dataset.map(lambda x: {"payload": unquote(x["payload"], encoding='latin1')})

Map:   0%|          | 0/17913 [00:00<?, ? examples/s]

Map:   0%|          | 0/7677 [00:00<?, ? examples/s]

In [ ]:
sample = csic_dataset["train"].shuffle(seed=42).select(range(10))

# Peek at the first few examples
sample[:5]

{'method': ['POST', 'POST', 'POST', 'POST', 'POST'],
 'payload': ['modo=insertar&precioA=1202&B1=Pasar+por+caja',
  'errorMsgA=Credenciales+incorrectas',
  "modo=insertar','0','0','0','0');waitfor+delay+'0:0:15';--&precio=2018&B1=Pasar+por+caja",
  'modo=entrar&login=marion&pwd=sescun3ia&remember=on&B1=Entrar',
  'modo=registro&login=fergusso&password=depravador&nombre=Brigitte&apellidos=Verdugo+Olalla&email=bideau@fulanitoymenganito.eh&dni=99997973E&direccion=Valencia,+79+12?H&ciudad=Narrillos+del+Álamo&cp=34239&provincia=Málaga&ntc=5564897923459190&B1=Registrar'],
 'label': ['Anomalous', 'Anomalous', 'Anomalous', 'Normal', 'Normal']}

In [ ]:
csic_dataset = csic_dataset.map(lambda x: {"payload": x["payload"].lower()})

Map:   0%|          | 0/17913 [00:00<?, ? examples/s]

Map:   0%|          | 0/7677 [00:00<?, ? examples/s]

In [ ]:
sample = csic_dataset["train"].shuffle(seed=42).select(range(10))

# Peek at the first few examples
sample[:5]

{'method': ['POST', 'POST', 'POST', 'POST', 'POST'],
 'payload': ['modo=insertar&precioa=1202&b1=pasar+por+caja',
  'errormsga=credenciales+incorrectas',
  "modo=insertar','0','0','0','0');waitfor+delay+'0:0:15';--&precio=2018&b1=pasar+por+caja",
  'modo=entrar&login=marion&pwd=sescun3ia&remember=on&b1=entrar',
  'modo=registro&login=fergusso&password=depravador&nombre=brigitte&apellidos=verdugo+olalla&email=bideau@fulanitoymenganito.eh&dni=99997973e&direccion=valencia,+79+12?h&ciudad=narrillos+del+álamo&cp=34239&provincia=málaga&ntc=5564897923459190&b1=registrar'],
 'label': ['Anomalous', 'Anomalous', 'Anomalous', 'Normal', 'Normal']}

In [ ]:
csic_dataset = csic_dataset.map(lambda x: {"payload": x["payload"].replace('=', ' ').replace('&', ' ')})

Map:   0%|          | 0/17913 [00:00<?, ? examples/s]

Map:   0%|          | 0/7677 [00:00<?, ? examples/s]

In [ ]:
sample = csic_dataset["train"].shuffle(seed=42).select(range(10))

# Peek at the first few examples
sample[:5]

{'method': ['POST', 'POST', 'POST', 'POST', 'POST'],
 'payload': ['modo insertar precioa 1202 b1 pasar+por+caja',
  'errormsga credenciales+incorrectas',
  "modo insertar','0','0','0','0');waitfor+delay+'0:0:15';-- precio 2018 b1 pasar+por+caja",
  'modo entrar login marion pwd sescun3ia remember on b1 entrar',
  'modo registro login fergusso password depravador nombre brigitte apellidos verdugo+olalla email bideau@fulanitoymenganito.eh dni 99997973e direccion valencia,+79+12?h ciudad narrillos+del+álamo cp 34239 provincia málaga ntc 5564897923459190 b1 registrar'],
 'label': ['Anomalous', 'Anomalous', 'Anomalous', 'Normal', 'Normal']}

In [ ]:
def compute_payload_length(example):
    return {"payload_length": len(example["payload"].split())}

In [ ]:
csic_dataset = csic_dataset.map(compute_payload_length)
# Inspect the first training example
csic_dataset["train"][0]

Map:   0%|          | 0/17913 [00:00<?, ? examples/s]

Map:   0%|          | 0/7677 [00:00<?, ? examples/s]

{'method': 'POST',
 'payload': 'modo registro login snips password impenetr-abilidad nombre kalanit apellidos pinós+turrubiartes email ornellas3@professionalhost.com.pro dni 98057218f direccion calle+batalla+de+otumba+26,+ ciudad lora+de+estepa cp 43120 provincia salamanca ntc 3586735536952135 b1 registrar',
 'label': 'Normal',
 'payload_length': 26}

In [ ]:
csic_dataset["train"].sort("payload_length")[:3]

{'method': ['POST', 'POST', 'POST'],
 'payload': ['id 1', "id ','0','0');waitfor+delay+'0:0:15';--", 'id 2'],
 'label': ['Normal', 'Anomalous', 'Normal'],
 'payload_length': [2, 2, 2]}

In [ ]:
csic_dataset = csic_dataset.filter(lambda x: x["payload_length"] > 6)
print(csic_dataset.num_rows)

Filter:   0%|          | 0/17913 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7677 [00:00<?, ? examples/s]

{'train': 9480, 'test': 4129}


In [ ]:
csic_dataset_clean = csic_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
csic_dataset_clean["validation"] = csic_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
csic_dataset_clean["test"] = csic_dataset["test"]
csic_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['method', 'payload', 'label', 'payload_length'],
        num_rows: 7584
    })
    validation: Dataset({
        features: ['method', 'payload', 'label', 'payload_length'],
        num_rows: 1896
    })
    test: Dataset({
        features: ['method', 'payload', 'label', 'payload_length'],
        num_rows: 4129
    })
})

In [ ]:
# list(set(csic_dataset_clean["validation"]['label']))

In [ ]:
# !rm -fr csic-dataset

In [ ]:
# csic_dataset_clean.save_to_disk("csic-dataset")

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# csic_dataset_clean.push_to_hub("csic-dataset")

### Load Dataset

In [ ]:
# !rm -fr csic-dataset/

In [ ]:
# dataset = load_dataset("b1nch3f/csic-dataset")
dataset = csic_dataset_clean
dataset

DatasetDict({
    train: Dataset({
        features: ['method', 'payload', 'label', 'payload_length'],
        num_rows: 7584
    })
    validation: Dataset({
        features: ['method', 'payload', 'label', 'payload_length'],
        num_rows: 1896
    })
    test: Dataset({
        features: ['method', 'payload', 'label', 'payload_length'],
        num_rows: 4129
    })
})

In [ ]:
dataset["train"][0]

{'method': 'POST',
 'payload': "modo registro login nettle password heñir nombre rad';+drop+table+usuarios;+select+*+from+datos+where+nombre+like+'% apellidos zalarich+llufriu email duarte@ingenieriadelaconstruccion.vc dni 47182191z direccion baza,+55,+ ciudad blecua+y+torres cp 43151 provincia granada ntc 8375962445368254 b1 registrar",
 'label': 'Anomalous',
 'payload_length': 26}

In [ ]:
def encode_label(example):
    return {"encode_label": 1 if example["label"] == 'Anomalous' else 0}

In [ ]:
dataset = dataset.map(encode_label)

Map:   0%|          | 0/7584 [00:00<?, ? examples/s]

Map:   0%|          | 0/1896 [00:00<?, ? examples/s]

Map:   0%|          | 0/4129 [00:00<?, ? examples/s]

In [ ]:
dataset["train"][0]

{'method': 'POST',
 'payload': "modo registro login nettle password heñir nombre rad';+drop+table+usuarios;+select+*+from+datos+where+nombre+like+'% apellidos zalarich+llufriu email duarte@ingenieriadelaconstruccion.vc dni 47182191z direccion baza,+55,+ ciudad blecua+y+torres cp 43151 provincia granada ntc 8375962445368254 b1 registrar",
 'label': 'Anomalous',
 'payload_length': 26,
 'encode_label': 1}

In [ ]:
dataset = dataset.remove_columns(["label"])

In [ ]:
dataset = dataset.rename_column("encode_label", "label")

In [ ]:
dataset["train"][0]

{'method': 'POST',
 'payload': "modo registro login nettle password heñir nombre rad';+drop+table+usuarios;+select+*+from+datos+where+nombre+like+'% apellidos zalarich+llufriu email duarte@ingenieriadelaconstruccion.vc dni 47182191z direccion baza,+55,+ ciudad blecua+y+torres cp 43151 provincia granada ntc 8375962445368254 b1 registrar",
 'payload_length': 26,
 'label': 1}

In [ ]:
list(set(dataset["train"]['label']))

[0, 1]

As you now know, you need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. To process your dataset in one step, use 🤗 Datasets [`map`](https://huggingface.co/docs/datasets/process.html#map) method to apply a preprocessing function over the entire dataset:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["payload"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5406 [00:00<?, ? examples/s]

Map:   0%|          | 0/1352 [00:00<?, ? examples/s]

Map:   0%|          | 0/2849 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['method', 'payload', 'payload_length', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5406
    })
    validation: Dataset({
        features: ['method', 'payload', 'payload_length', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1352
    })
    test: Dataset({
        features: ['method', 'payload', 'payload_length', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2849
    })
})

If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes:

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# small_train_dataset = tokenized_datasets["train"]
# small_eval_dataset = tokenized_datasets["test"]

## Train

## Train with PyTorch Trainer

🤗 Transformers provides a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) class optimized for training 🤗 Transformers models, making it easier to start training without manually writing your own training loop. The [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) API supports a wide range of training options and features such as logging, gradient accumulation, and mixed precision.

Start by loading your model and specify the number of expected labels. From the Yelp Review [dataset card](https://huggingface.co/datasets/yelp_review_full#data-fields), you know there are five labels:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<Tip>

You will see a warning about some of the pretrained weights not being used and some weights being randomly
initialized. Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it.

</Tip>

### Training hyperparameters

Next, create a [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training [hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments), but feel free to experiment with these to find your optimal settings.

Specify where to save the checkpoints from your training:

In [ ]:
!rm -fr test_trainer

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

### Evaluate

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) does not automatically evaluate model performance during training. You'll need to pass [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) a function to compute and report metrics. The [🤗 Evaluate](https://huggingface.co/docs/evaluate/index) library provides a simple [`accuracy`](https://huggingface.co/spaces/evaluate-metric/accuracy) function you can load with the [evaluate.load](https://huggingface.co/docs/evaluate/main/en/package_reference/loading_methods#evaluate.load) (see this [quicktour](https://huggingface.co/docs/evaluate/a_quick_tour) for more information) function:

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Call `compute` on `metric` to calculate the accuracy of your predictions. Before passing your predictions to `compute`, you need to convert the predictions to logits (remember all 🤗 Transformers models return logits):

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

If you'd like to monitor your evaluation metrics during fine-tuning, specify the `evaluation_strategy` parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
)

### Trainer

Create a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.595008,0.660000
2,No log,0.578030,0.720000
3,No log,0.363729,0.828000
4,0.451600,0.439840,0.876000
5,0.451600,0.439549,0.883000


TrainOutput(global_step=625, training_loss=0.3969122314453125, metrics={'train_runtime': 223.0865, 'train_samples_per_second': 22.413, 'train_steps_per_second': 2.802, 'total_flos': 1315555276800000.0, 'train_loss': 0.3969122314453125, 'epoch': 5.0})

## Train in native PyTorch

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) takes care of the training loop and allows you to fine-tune a model in a single line of code. For users who prefer to write their own training loop, you can also fine-tune a 🤗 Transformers model in native PyTorch.

At this point, you may need to restart your notebook or execute the following code to free some memory:

In [ ]:
import torch

In [ ]:
# del model
# del trainer
torch.cuda.empty_cache()

Next, manually postprocess `tokenized_dataset` to prepare it for training.

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["payload", "method", "payload_length"])

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [ ]:
tokenized_datasets.set_format("torch")

Then create a smaller subset of the dataset as previously shown to speed up the fine-tuning:

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

### DataLoader

Create a `DataLoader` for your training and test datasets so you can iterate over batches of data:

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Load your model with the number of expected labels:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Optimizer and learning rate scheduler

Create an optimizer and learning rate scheduler to fine-tune the model. Let's use the [`AdamW`](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) optimizer from PyTorch:

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

Create the default learning rate scheduler from [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer):

In [ ]:
from transformers import get_scheduler

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

Lastly, specify `device` to use a GPU if you have access to one. Otherwise, training on a CPU may take several hours instead of a couple of minutes.

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### Training loop

To keep track of your training progress, use the [tqdm](https://tqdm.github.io/) library to add a progress bar over the number of training steps:

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/625 [00:00<?, ?it/s]

### Evaluate

Just like how you added an evaluation function to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), you need to do the same when you write your own training loop. But instead of calculating and reporting the metric at the end of each epoch, this time you'll accumulate all the batches with `add_batch` and calculate the metric at the very end.

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.591}